<h1>Data Wrangling with Regular Expressions</h1>




To summarize, we will be using `stringr` (part of `tidyverse`) and regular expressions to perform the following data wrangling tasks:

*   Standardize column names for all collected datasets
*   Remove undesired reference links from the scraped bike-sharing systems dataset
*   Extract only the numeric value from undesired text annotations


Let's begin by importing the libraries


In [1]:
# Check whether you need to install the `tidyverse` library
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


## TASK: Standardize column names for all collected datasets




*   Column names need to be UPPERCASE
*   The word separator needs to be an underscore, such as in `COLUMN_NAME`


we can use the following dataset list and the `names()` function to get and set each of their column names, and convert them according to our defined naming convention.


In [2]:
dataset_list <- c('bike_sharing_systems.csv', 'raw_seoul_bike_sharing.csv', 'cities_weather_forecast.csv', 'raw_worldcities.csv')

 `for` loop to iterate over the above datasets and convert their column names


In [3]:
for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name)
    # Standardized its columns:
    # Convert all column names to uppercase
    names(dataset) <- toupper(names(dataset))
    # Replace any white space separators by underscores, using the str_replace_all function
    names(dataset) <- str_replace_all(names(dataset)," ", "_")
    # Save the dataset 
    write.csv(dataset, dataset_name, row.names=FALSE)
}


Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_double(),
  Country = col_character(),
  City = col_character(),
  Name = col_character(),
  System = col_character(),
  Operator = col_character(),
  Launched = col_character(),
  Discontinued = col_character(),
  Stations = col_character(),
  Bicycles = col_character(),
  `Daily ridership` = col_character()
)
Parsed with column specification:
cols(
  Date = col_character(),
  RENTED_BIKE_COUNT = col_double(),
  Hour = col_double(),
  TEMPERATURE = col_double(),
  HUMIDITY = col_double(),
  WIND_SPEED = col_double(),
  Visibility = col_double(),
  DEW_POINT_TEMPERATURE = col_double(),
  SOLAR_RADIATION = col_double(),
  RAINFALL = col_double(),
  Snowfall = col_double(),
  SEASONS = col_character(),
  HOLIDAY = col_character(),
  FUNCTIONING_DAY = col_character()
)
Parsed with column specification:
cols(
  city = col_character(),
  weather = col_character(),
  visibility = col

Read the resulting datasets back and check whether their column names follow the naming convention


In [4]:
for (dataset_name in dataset_list){
    # Print a summary for each data set to check whether the column names were correctly converted
    print(summary(dataset))
}

     CITY            CITY_ASCII             LAT              LNG           
 Length:26569       Length:26569       Min.   :-54.93   Min.   :-179.5900  
 Class :character   Class :character   1st Qu.: 27.92   1st Qu.: -78.7794  
 Mode  :character   Mode  :character   Median : 40.22   Median :  -0.7689  
                                       Mean   : 33.10   Mean   : -11.3639  
                                       3rd Qu.: 47.99   3rd Qu.:  29.6833  
                                       Max.   : 81.72   Max.   : 179.3667  
                                                                           
   COUNTRY              ISO2               ISO3            ADMIN_NAME       
 Length:26569       Length:26569       Length:26569       Length:26569      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
       

## Process the web-scraped bike sharing system dataset


By now we have standardized all column names. Next, we will focus on cleaning up the values in the web-scraped bike sharing systems dataset.


In [5]:
# First load the dataset
bike_sharing_df <- read_csv("bike_sharing_systems.csv")

Parsed with column specification:
cols(
  X1 = col_double(),
  COUNTRY = col_character(),
  CITY = col_character(),
  NAME = col_character(),
  SYSTEM = col_character(),
  OPERATOR = col_character(),
  LAUNCHED = col_character(),
  DISCONTINUED = col_character(),
  STATIONS = col_character(),
  BICYCLES = col_character(),
  DAILY_RIDERSHIP = col_character()
)


In [6]:
# Print its head
head(bike_sharing_df)

X1,COUNTRY,CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
1,Albania,Tirana[5],Ecovolis,NA,NA,March 2011,NA,8,200,NA
2,Argentina,Mendoza[6],Metrobici,NA,NA,2014,NA,2,40,NA
3,Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,NA,27 November 2016,NA,8,80,NA
4,Argentina,Buenos Aires[7][8],Ecobici,Serttel Brasil[9],Bike In Baires Consortium.[10],2010,NA,400,4000,21917
5,Argentina,Rosario,Mi Bici Tu Bici[11],NA,NA,2 December 2015,NA,47,480,NA
6,Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,NA


Even from the first few rows, you can see there is plenty of undesireable embedded textual content, such as the reference link included in `Melbourne[12]`.


let's only focus on processing the following revelant columns :

*   `COUNTRY`: Country name
*   `CITY`: City name
*   `SYSTEM`: Bike-sharing system name
*   `BICYCLES`: Total number of bikes in the system


In [7]:
# Select the four columns
sub_bike_sharing_df <- bike_sharing_df %>% select(COUNTRY, CITY, SYSTEM, BICYCLES)

Let's see the types of the selected columns


In [8]:
sub_bike_sharing_df %>% 
    summarize_all(class) %>%
    gather(variable, class)

variable,class
COUNTRY,character
CITY,character
SYSTEM,character
BICYCLES,character


They are all interpreted as character columns, but we expect the `BICYCLES` column to be of numeric type. Let's see why it wasn't loaded as a numeric column - possibly some entries contain characters. Let's create a simple function called `find_character` to check that.


In [9]:
# grepl searches a string for non-digital characters, and returns TRUE or FALSE
# if it finds any non-digital characters, then the bicyle column is not purely numeric
find_character <- function(strings) grepl("[^0-9]", strings)

Let's try to find any elements in the `Bicycles` column containing non-numeric characters.


In [10]:
sub_bike_sharing_df %>% 
    select(BICYCLES) %>% 
    filter(find_character(BICYCLES)) %>%
    slice(0:10)

BICYCLES
1790 (2019)[21]
4200 (2021)
4115[26]
310[63]
500[76]
[79]
180[80]
600[81]
[82]
initially 800 (later 2500)


As you can see, many rows have non-numeric characters, such as `32 (including 6 rollers) [162]` and `1000[253]`. This is actually very common for a table scraped from Wiki when no input validation is enforced.

Later, you will use regular expressions to clean them up.


Next, let's take a look at the other columns, namely `COUNTRY`, `CITY`, and `SYSTEM`, to see if they contain any undesired reference links, such as in `Melbourne[12]`.


In [11]:
# Define a 'reference link' character class, 
# `[A-z0-9]` means at least one character 
# `\\[` and `\\]` means the character is wrapped by [], such as for [12] or [abc]
ref_pattern <- "\\[[A-z0-9]+\\]"
find_reference_pattern <- function(strings) grepl(ref_pattern, strings)

In [12]:
# Check whether the COUNTRY column has any reference links
sub_bike_sharing_df %>% 
    select(COUNTRY) %>% 
    filter(find_reference_pattern(COUNTRY)) %>%
    slice(0:10)

COUNTRY


Ok, looks like the `COUNTRY` column is clean. Let's check the `CITY` column.


In [13]:
# Check whether the CITY column has any reference links
sub_bike_sharing_df %>% 
    select(CITY) %>% 
    filter(find_reference_pattern(CITY)) %>%
    slice(0:10)

CITY
Tirana[5]
Mendoza[6]
Buenos Aires[7][8]
Melbourne[12]
Brisbane[14][15]
Lower Austria[18]
Different locations[19]
Namur[22]
Brussels[25]
Salvador[27]


Hmm, looks like the `CITY` column has some reference links to be removed. Next, let's check the `SYSTEM` column.


In [14]:
# Check whether the System column has any reference links
sub_bike_sharing_df %>% 
    select(SYSTEM) %>% 
    filter(find_reference_pattern(SYSTEM)) %>%
    slice(0:10)

SYSTEM
Serttel Brasil[9]
EasyBike[62]
4 Gen.[65]
3 Gen. SmooveKey[118]
3 Gen. Smoove[146][147][148][144]
3 Gen. Smoove[184]
3 Gen. Smoove[186]
3 Gen. Smoove[188]


So the `SYSTEM` column also has some reference links.


After some preliminary investigations, we identified that the `CITY` and `SYSTEM` columns have some undesired reference links, and the `BICYCLES` column has both reference links and some
textual annotations.

Next, you need to use regular expressions to clean up the unexpected reference links and text annotations in numeric values.


## Remove undesired reference links using regular expressions


Write a custom function using `stringr::str_replace_all` to replace all reference links with an empty character for columns `CITY` and `SYSTEM`


In [15]:
# remove reference link
remove_ref <- function(strings) {
    ref_pattern <- "\\[[A-z0-9]+\\]"
    # Replace all matched substrings with a white space using str_replace_all()
    result=str_replace_all(strings, ref_pattern,"")
    # Trim the reslt if you want
    return(result)
}

 Use the `dplyr::mutate()` function to apply the `remove_ref` function to the `CITY` and `SYSTEM` columns


In [16]:
result=sub_bike_sharing_df %>% 
mutate(CITY=remove_ref(CITY), SYSTEM=remove_ref(SYSTEM) )

 Use the following code to check whether all reference links are removed:


In [17]:
result %>% 
    select(CITY, SYSTEM, BICYCLES) %>% 
    filter(find_reference_pattern(CITY) | find_reference_pattern(SYSTEM) | find_reference_pattern(BICYCLES))

CITY,SYSTEM,BICYCLES
Different locations,Blue-bike,1790 (2019)[21]
Brussels,3 Gen. Cyclocity,4115[26]
"Limassol (& Agios Dometios, Aglandjia, Dali, Engomi, Latsia, Pallouriotissa, Strovolos)",3 Gen. Smoove,310[63]
Prague,NA,500[76]
Prague 7,4 Gen. Ofo,[79]
Prostejov,3 Gen. nextbike,180[80]
Ostrava,3 Gen. nextbike,600[81]
Fars<f8>,2 Gen,[82]
Batumi,3 Gen. SmooveKey,370[119]
Darmstadt,3 & 4 Gen. Call a Bike flex,350 [130]


# Extract the numeric value using regular expressions


 Write a custom function using `stringr::str_extract` to extract the first digital substring match and convert it into numeric type For example, extract the value '32' from `32 (including 6 rollers) [162]`.


In [18]:
# Extract the first number
extract_num <- function(columns){
    # Define a digital pattern
    digitals_pattern <- "[0-9]+"
    # Find the first match using str_extract
    result=str_extract(columns, digitals_pattern)
    # Convert the result to numeric using the as.numeric() function
    as.numeric(result)
}

 Use the `dplyr::mutate()` function to apply `extract_num` on the `BICYCLES` column


In [19]:
# Use the mutate() function on the BICYCLES column
result =sub_bike_sharing_df %>%
mutate(BICYCLES=extract_num(BICYCLES))

 Use the summary function to check the descriptive statistics of the numeric `BICYCLES` column


In [20]:
summary(result$BICYCLES)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
      5      94     300    1921    1250   78000      78 

 Write the cleaned bike-sharing systems dataset into a csv file called `bike_sharing_systems.csv`


In [21]:
# Write dataset to `bike_sharing_systems.csv`
write.csv(sub_bike_sharing_df, 'bike_sharing_systems.csv')